## 整理数据集

把数据集放到 datases/raw/ 目录下，建两个文件夹 y 和 n

- datasets/raw/y/ 里面放技能 ready 的图
- datasets/raw/n/ 里面放没技能的图

要求输入图片尺寸为 64x64x3，文件名随便

这段代码主要是定义了一个用于整理技能数据集的类 `SkillDataset`，并使用该类实例化了一个数据集 `dataset`。
- `pathlib.Path` 是一个操作路径的类。 `Path("datasets/raw")` 创建一个指向 `datasets/raw` 目录的 `Path` 对象。
- `default_loader` 是一个函数，用于加载图片。它接收一个路径，返回该路径指向的图片经过处理后的结果。

`SkillDataset` 是一个类，继承自 `Dataset`，用于整理技能数据集。它有一个构造函数 `__init__`，接收一个 `path` 参数，即数据集所在的路径。
- `self.y` 是一个列表，包含所有技能数据集中“有技能”的图片的路径。`self.n` 是一个列表，包含所有技能数据集中“无技能”的图片的路径。
- `self.transform` 是一个 `torchvision.transforms` 的实例，用于对图片进行预处理，包括高斯模糊、随机压缩、锐化、自动对比度、转化为张量。
- `self.loader` 是一个函数，用于加载图片。
- `self.data` 是一个列表，用于存储数据集中所有图片的处理结果。
- `__len__` 是一个特殊函数，用于获取数据集的长度。这里返回的是“有技能”图片和“无技能”图片的数量之和。
- `get` 函数用于获取指定索引的图片和标签。如果索引小于“有技能”图片的数量，那么这张图片就是“有技能”的，返回的标签为 1；否则这张图片就是“无技能”的，返回的标签为 0。
- `__getitem__` 是一个特殊函数，用于获取指定索引的图片和标签。

In [ ]:
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

dataset_path = Path("datasets/raw")

def default_loader(path):
    return Image.open(path).convert('RGB')

class SkillDataset(Dataset):
    def __init__(self, path: Path) -> None:
        super().__init__()
        self.y = list((path / 'y').glob('**/*'))
        self.n = list((path / 'n').glob('**/*'))
        self.transform = transforms.Compose([
            transforms.GaussianBlur(3, sigma=(0.1, 2.0)), # 高斯模糊，卷积核的大小为3x3，水平和垂直方向上的标准偏差分别为 0.1 和 2.0
            transforms.RandomPosterize(3),                # 随机压缩
            transforms.RandomAdjustSharpness(3),          # 随机锐化
            transforms.RandomAutocontrast(),              # 自动对比度调整
            transforms.ToTensor(),                        # 将图像转换为 PyTorch 张量
        ])
        self.loader = default_loader
        # 技能图片没多大，一次性全部载入内存算了
        self.data = [ self.get(i) for i in range(len(self))]
    
    def __len__(self):
        return len(self.y) + len(self.n)
    
    def get(self, index):
        if index < len(self.y):
            if index % 1000 == 0:
                print(f'load y: {index} / {len(self.y)}')
            path = self.y[index]
            label = 1
        else:
            if index % 1000 == 0:
                print(f'load n: {index - len(self.y)} / {len(self.n)}')
            path = self.n[index - len(self.y)]
            label = 0
        image = self.loader(path)
        image = self.transform(image)
        return image, label
    
    def __getitem__(self, index):
        return self.data[index]

dataset = SkillDataset(dataset_path)

这是一个 `PyTorch` 的数据处理部分的代码片段，主要实现了数据的划分和加载。

- 该代码将数据集划分为训练集和测试集，其中训练集占总数据集的 80%，测试集占 20%。
- 然后使用 `DataLoader` 对训练集和测试集进行批处理，每次处理的批次大小分别为 4096 和 2048，并进行随机打乱（对于训练集），不打乱（对于测试集）。
- 其中 `num_workers` 参数表示数据读取的并行进程数量，这里设置为 0 表示不使用多线程读取数据。

如果显存不够，可以尝试减小批次大小，以减少显存的占用。

In [ ]:
import torch

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False, num_workers=0)

这个代码实现了一个简单的GoogleNet模型，其中包含了两个Inception模块。

Inception模块是GoogleNet中的重要组成部分，通过使用不同大小和类型的卷积核来并行计算特征图，从而提高了模型的精度和效率。

GoogleNet是经典的深度神经网络，可以用于图像分类、目标检测和语义分割等任务。

In [ ]:
import torch

class InceptionA(torch.nn.Module):
    def __init__(self, in_ch) -> None:
        super().__init__()
        self.branch_1x1 = torch.nn.Conv2d(in_ch, 16, kernel_size=1)

        self.branch_5x5_1 = torch.nn.Conv2d(in_ch, 16, kernel_size=1)
        self.branch_5x5_2 = torch.nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch_3x3_1 = torch.nn.Conv2d(in_ch, 16, kernel_size=1)
        self.branch_3x3_2 = torch.nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch_3x3_3 = torch.nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = torch.nn.Conv2d(in_ch, 24, kernel_size=1)

    def forward(self, x):
        branch_1x1 = self.branch_1x1(x)

        branch_5x5 = self.branch_5x5_1(x)
        branch_5x5 = self.branch_5x5_2(branch_5x5)

        branch_3x3 = self.branch_3x3_1(x)
        branch_3x3 = self.branch_3x3_2(branch_3x3)
        branch_3x3 = self.branch_3x3_3(branch_3x3)

        branch_pool = torch.nn.functional.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch_1x1, branch_5x5, branch_3x3, branch_pool] # 16 + 24 + 24 + 24
        return torch.cat(outputs, 1)


class GoogleNet(torch.nn.Module):
    def __init__(self, channels) -> None:
        super().__init__()
        self.conv1 = torch.nn.Conv2d(channels, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(88, 20, kernel_size=5)
        self.incep1 = InceptionA(10)
        self.incep2 = InceptionA(20)
        self.mp = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(14872, 2)

    def forward(self, x):
        in_size = x.size(0)
        x = torch.nn.functional.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = torch.nn.functional.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x


这段代码使用 PyTorch 的自动混合精度加速训练过程。

自动混合精度可以让模型的训练更快，同时避免数值下溢或上溢的问题。具体而言，它可以将 `float32` 数据类型转换为 `float16`，以减少 GPU 内存的使用量，从而可以使用更大的 `batch size` 加速训练。同时，使用 `GradScaler` 可以帮助我们调整梯度的缩放因子，以保持数值稳定性。

这里的代码也检查了系统是否支持 CUDA，如果不支持，则会在 CPU 上进行训练。

In [ ]:
from torch.cuda.amp import autocast, GradScaler

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if not use_cuda:
    print("WARNING: CPU will be used for training.")

model = GoogleNet(3).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scaler = GradScaler()

这段代码是定义了训练和测试模型的函数。

`train` 函数用于训练模型。在函数中，将模型设置为训练模式，并遍历训练集，将数据和目标标签分别移动到设备上进行计算。然后，通过 `optimizer.zero_grad()` 清除之前的梯度，并在 `autocast()` 内使用自动混合精度进行前向传递、计算损失和反向传递。最后，使用 `scaler.step()` 进行优化步骤，并使用 `scaler.update()` 更新缩放器以便下一个批次的使用。最后，打印出损失和时间。

`test` 函数用于测试模型。在函数中，将模型设置为评估模式，并遍历测试集，将数据和目标标签分别移动到设备上进行计算。通过 `model(data)` 计算模型的输出，并计算测试集上的损失和准确率。最后，打印出测试集上的损失和准确率，并返回这两个值。

In [ ]:
import time

start_time = time.time()
def train(epoch):
    global start_time
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        with autocast():
            output = model(data)
            loss = criterion(output, target)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
    cur_time = time.time()
    cost = cur_time - start_time
    print(f'Train Epoch: {epoch}, Loss: {loss.item():.8f}, cost: {cost:.2f} s')
    start_time = cur_time
            
def test():
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            with autocast():
                output = model(data)
                test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)

    print(f'=== Test: Loss: {test_loss:.8f}, Acc: {acc:.4f} ===')
    return test_loss, acc

这段代码是训练模型的主要过程。

在训练过程中，每个 epoch 会先调用 `train` 函数来训练模型，然后调用 `test` 函数来评估模型在测试集上的性能。在每 `test_interval`（默认为10）个 epoch 后，会评估模型在测试集上的性能，并记录当前的最佳模型。

如果当前 epoch 的测试集上的损失大于当前最佳损失，则认为模型性能没有改进。如果当前 epoch 与最佳 epoch 之间的差距超过10个 `test_interval` ，则认为模型已经很久没有得到改进，训练将被中止。如果当前 epoch 的测试集上的损失小于当前最佳损失，则将当前模型保存为最佳模型，并更新最佳 epoch 、最佳损失和最佳准确率的值。

最后，训练结束后，最佳模型将保存在 `checkpoints` 目录下，文件名为 `GoogleNet_best.pt`。

In [ ]:
if use_cuda:
    torch.cuda.empty_cache()

output = Path('checkpoints')
output.mkdir(exist_ok=True, parents=True)

model_name = model.__class__.__name__
best_model_path = output / f'{model_name}_best.pt'

best_epoch = 0
best_loss = 100.0
best_acc = 0.0
default_interval = 1

def pipeline(start_epoch = 0, test_interval = default_interval):
    global best_epoch, best_loss, best_acc

    for epoch in range(start_epoch, 1000):
        train(epoch)
        if epoch % test_interval != 0:
            continue
        
        loss, acc = test()
        print(f'=== Pre best is {best_epoch}, Loss: {best_loss:.8f}, Acc: {best_acc:.4f} ===')
        torch.save(model, output / f'{model_name}_{epoch}.pt')
        if loss > best_loss:
            if epoch - best_epoch > 100:
                print('No improvement for a long time, Early stop!')
                break
            else:
                continue
        best_epoch = epoch
        best_loss = loss
        best_acc = acc
        print(f'====== New best is {best_epoch}, Loss: {best_loss:.8f}, Acc: {best_acc:.4f} ======')
        torch.save(model, best_model_path)

pipeline()

这段代码会从文件中加载最佳模型，并在测试集上进行评估。然后它将返回测试集上的损失和准确率。

In [ ]:
model = torch.load(best_model_path)
test()

这段代码是将 `PyTorch` 训练好的模型导出为 `ONNX` 格式，以便在其他框架或硬件上进行部署和推理。主要包含了以下几个步骤：

1. 加载训练好的模型：使用 `torch.load` 函数加载PyTorch模型，`map_location=torch.device("cpu")`表示将模型参数加载到 CPU 上。
2. 设置输入：使用 `torch.randn` 函数创建一个随机的输入数据，用于设置模型的输入大小。
3. 导出模型：使用 `torch.onnx.export` 函数将模型导出为 `ONNX` 格式，需要指定模型、输入数据、输出文件路径、输入和输出的名称。
4. 值得注意的是，在导出 `ONNX` 模型时，需要注意模型的输入输出名称、大小和顺序等信息，以确保在后续的推理中能够正确地加载和使用模型。

In [ ]:
import torch.onnx
from pathlib import Path


def convert_onnx(path: Path):
    model = torch.load(path, map_location=torch.device("cpu"))
    model.eval()
    dummy_input = torch.randn(1, 3, 64, 64)
    torch.onnx.export(
        model,
        dummy_input,
        path.with_suffix(".onnx"),
        input_names=["input"],
        output_names=["output"],
    )


convert_onnx(best_model_path)


该代码是用于检查 `ONNX` 模型是否规范。如果检查失败，将引发异常并显示有关问题的详细信息。

In [ ]:
import onnx

onnx.checker.check_model(str(best_model_path.with_suffix(".onnx")))

## 以下是部分清洗数据的代码，可以自己研究下

由于是注释掉的代码，并且需要手工清洗，所以我们不能直接运行，需要手动选择和清洗一些数据。

这部分代码的目的是从原始数据集中随机选择一定数量的“正面”和“负面”图像样本，用于构建清洗后的数据集。这里假设原始数据集是由两个子目录“y”和“n”组成，每个子目录包含正面或负面的样本图片。

这部分代码的思路是首先指定要从原始数据集中选择多少个样本（`clean_set_size`），然后随机从两个子目录中选择相同数量的样本，并将其移动到另一个目录中（`clean_y`或`clean_n`）作为清洗后的数据集。需要手工查看每个样本并判断其是否属于相应的类别。

In [ ]:
# 随机数据挑选，请手工清洗

# from pathlib import Path
# import random

# clean_set_size = 1000
# raw_path = Path("datasets/raw")
# positive_set = random.sample(list((raw_path / "y").glob("**/*")), clean_set_size)
# negative_set = random.sample(list((raw_path / "n").glob("**/*")), clean_set_size)

# clean = Path("datasets/clean/")
# clean_y = clean / "y"
# clean_y.mkdir(parents=True, exist_ok=True)
# clean_n = clean / "n"
# clean_n.mkdir(parents=True, exist_ok=True)
# for path in positive_set:
#     path.rename(clean_y / path.name)
# for path in negative_set:
#     path.rename(clean_n / path.name)


与上文中的 `SkillDataset` 类似，定义了一个名为 `SkillRawDataset` 的数据集类

In [ ]:
# class SkillRawDataset(Dataset):
#     def __init__(self) -> None:
#         super().__init__()
#         self.y = list(Path('datasets/raw/y/').glob('**/*'))
#         self.n = list(Path('datasets/raw/n/').glob('**/*'))
#         self.transform = transforms.ToTensor()
#         self.loader = default_loader
#         self.data = [ self.get(i) for i in range(len(self))]
    
#     def __len__(self):
#         return len(self.y) + len(self.n)
    
#     def get(self, index):
#         # print(f'load: {self.count} / {len(self)}')
#         if index < len(self.y):
#             path = self.y[index]
#             label = 1
#         else:
#             path = self.n[index - len(self.y)]
#             label = 0
#         image = self.loader(path)
#         image = self.transform(image)
#         return image, label
    
#     def __getitem__(self, index):
#         return self.data[index]
    
#     def get_path(self, index):
#         if index < len(self.y):
#             path = self.y[index]
#             label = 1
#         else:
#             path = self.n[index - len(self.y)]
#             label = 0
#         return path, label

# raw_data_set = SkillRawDataset()


这段代码是对数据集进行清洗的代码，其中：

`SkillRawDataset` 是对原始数据集进行封装的 `Dataset` 类，其中 `__init__` 方法中会将 y 和 n 两个目录下的所有图片路径加载进来，并转换为数据和标签，存储到 `self.data` 中。`get` 方法用于获取指定 index 的数据和标签。
`raw_loader` 是使用 `SkillRawDataset` 加载数据集的 `DataLoader`，每个 batch 仅包含一个样本。
`clear` 函数用于遍历 `raw_loader` 中的每一个样本，并使用当前的模型进行前向推断，如果预测结果与标签不符，则将这张图片移到 `datasets/maybe_error/1` 或 `datasets/maybe_error/0` 目录下（根据标签确定）。

In [ ]:
# raw_loader = DataLoader(raw_data_set, batch_size=1, shuffle=False, num_workers=0)
# import os
# import shutil
# def clear():
#     model.eval()
#     test_loss = 0
#     Path('datasets/maybe_error/1').mkdir(parents=True, exist_ok=True)
#     Path('datasets/maybe_error/0').mkdir(parents=True, exist_ok=True)
#     with torch.no_grad():
#         for batch_idx, (data, target) in enumerate(raw_loader):
#             data, target = data.cuda(), target.cuda()
#             output = model(data)
#             loss = criterion(output, target).item() # sum up batch loss
#             test_loss += loss
#             pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
#             correct = pred.eq(target.view_as(pred)).sum().item()
#             if not correct:
#                 tup = raw_data_set.get_path(batch_idx)
#                 print(tup)
#                 os.rename(tup[0], Path('datasets/maybe_error/') / str(tup[1]) / tup[0].name)                
                


# print(len(raw_data_set))
# clear()